https://github.com/studioimaginaire/phue 

Phue is a package for interacting with the Hue hub. 

I have 2 remotes, 1 tap, and 3 motion (+ temp and light) sensors. Hue also creates 'sensors' for presence ('HomeAway', 'Robins iPhone') as well as dimmer secnes.

In [1]:
from phue import Bridge
from datetime import datetime, timedelta
from functools import wraps

First create dummy Throttle class stolen from https://gist.github.com/ChrisTM/5834503 as data updates will be throttled

In [2]:
class Throttle(object):
    """
    Decorator that prevents a function from being called more than once every
    time period.
    To create a function that cannot be called more than once a minute:
        @throttle(minutes=1)
        def my_fun():
            pass
    """
    def __init__(self, seconds=0, minutes=0, hours=0):
        self.throttle_period = timedelta(
            seconds=seconds, minutes=minutes, hours=hours
        )
        self.time_of_last_call = datetime.min

    def __call__(self, fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            now = datetime.now()
            time_since_last_call = now - self.time_of_last_call

            if time_since_last_call > self.throttle_period:
                self.time_of_last_call = now
                return fn(*args, **kwargs)

        return wrapper

# Setup variables

In [3]:
bridge_ip = '192.168.0.3'
SCAN_INTERVAL = 1   # Seconds

# Sensors
Sensors are more simple than lights in some ways, as lights can be in groups. A significant part of the HA lights component is dealing with groups. 

For SML motion sensors we have the issue that Hue treates these device as 3 seperate sensors consisting of motion, temperature and light level. I prefer to group by device so need to have a routine which correctly ties sensors to a device. The dimmer remote has scenes but will ignore these.

Lets create a wrapper to the bridge object that throttles requests for sensor data.

In [4]:
class HueSensorData(object):
    """Get the latest sensor data."""
    # need to import phue
    def __init__(self, bridge_ip):
        """Initialize the data object."""
        self._bridge = Bridge(bridge_ip)
        self.update()

    # Update only once in scan interval.
    @Throttle(seconds=SCAN_INTERVAL)
    def update(self):
        """Get the latest data but throttle requests."""
        self._data = self._bridge.get_sensor()
        
    @property
    def data(self):
        """Return the data."""
        return self._data

In [5]:
sensor_data_obj = HueSensorData(bridge_ip)

lets get a list of the sensor id available in data

In [7]:
sensor_id_list = list(sensor_data_obj.data.keys())
sensor_id_list

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '18',
 '20',
 '21',
 '22',
 '23',
 '24']

In [7]:
sensor_data_obj.data['10']

{'config': {'alert': 'none',
  'battery': 100,
  'ledindication': False,
  'on': True,
  'pending': [],
  'reachable': True,
  'tholddark': 12853,
  'tholdoffset': 7000,
  'usertest': False},
 'manufacturername': 'Philips',
 'modelid': 'SML001',
 'name': 'Hue ambient light sensor 2',
 'state': {'dark': True,
  'daylight': False,
  'lastupdated': '2018-01-18T08:14:50',
  'lightlevel': 4225},
 'swupdate': {'lastinstall': None, 'state': 'noupdates'},
 'swversion': '6.1.0.18912',
 'type': 'ZLLLightLevel',
 'uniqueid': '00:17:88:01:02:00:b5:ce-02-0400'}

In [11]:
for sensor_id in sensor_id_list:
    obj = sensor_data_obj.data[sensor_id]
    print("{} : modelid = {}".format(obj['name'], obj['modelid']))

Daylight : modelid = PHDL00
Remote bedroom : modelid = RWL021
Dimmer Switch 2 SceneCycle : modelid = PHWA01
Hue temperature sensor 1 : modelid = SML001
Hall Sensor : modelid = SML001
Hue ambient light sensor 1 : modelid = SML001
MotionSensor 5.Companion : modelid = PHA_STATE
Hue temperature sensor 2 : modelid = SML001
Bedroom sensor : modelid = SML001
Hue ambient light sensor 2 : modelid = SML001
MotionSensor 9.Companion : modelid = PHA_STATE
Robins iPhone : modelid = HA_GEOFENCE
HomeAway : modelid = HOMEAWAY
Hue temperature sensor 3 : modelid = SML001
Living room sensor : modelid = SML001
Hue ambient light sensor 3 : modelid = SML001
MotionSensor 15.Companion : modelid = PHA_STATE
Living room remote : modelid = RWL021
Dimmer Switch 20 SceneCycle : modelid = PHWA01
Hall remote : modelid = RWL021
Dimmer Switch 22 SceneCycle : modelid = PHWA01
Hue Tap : modelid = ZGPSWITCH


### Sensor device id

lets create a unique device id, where for a motion sensor the device includes both temperature and light level sensors

In [26]:
def device_id(sensor_obj):
    """Helper that takes a sensor object and returns its unique device id."""
    assert str(type(sensor_obj)) == "<class 'phue.Sensor'>"
    return sensor_obj.uniqueid.split(':')[-1][0:5]

In [13]:
device_id(response['Bedroom sensor'])

'ce-02'

In [14]:
device_id(response['Hue ambient light sensor 2'])

'ce-02'

In [15]:
device_id(response['Hue temperature sensor 2'])

'ce-02'

In [16]:
print_sensor_attrs('Hue ambient light sensor 2')

name : Hue ambient light sensor 2
state : {'lightlevel': 3433, 'dark': True, 'daylight': False, 'lastupdated': '2018-01-18T08:09:50'}
type : ZLLLightLevel
manufacturername : Philips
modelid : SML001
uniqueid : 00:17:88:01:02:00:b5:ce-02-0400
sensor_id : 10
config : {'on': True, 'battery': 100, 'reachable': True, 'alert': 'none', 'tholddark': 12853, 'tholdoffset': 7000, 'ledindication': False, 'usertest': False, 'pending': []}


So even though we have 1 device we have different names for its temperature and light level sensors. Associate these sensors by device_id. Lets create a HueSensor object.

In [17]:
class HueSensor(object):
    """Class to hold Hue Sensor basic info."""

    def __init__(self, sensor_obj):
        """Initialise the sensor object."""
        self._device_id = sensor_obj.uniqueid.split(':')[-1][0:5]
        self._data = data_obj    # data is in .data
        self._name = self._data.data[self._hue_id]['name']
        self._model = self._data.data[self._hue_id]['model']
        self._state = self._data.data[self._hue_id]['state']
        self._attributes = {}

id and names can be related using the method get_sensor_id_by_name()

In [18]:
b.get_sensor_id_by_name('Hall remote')

'22'

Lets check what response by id returns - sensors are indexed 1-24 as they appear listed from the sensors API return

In [19]:
response_id = b.get_sensor_objects('id')
response_id

{1: <phue.Sensor object "Daylight" at 0x10988d2b0>,
 2: <phue.Sensor object "Remote bedroom" at 0x10989e080>,
 3: <phue.Sensor object "Dimmer Switch 2 SceneCycle" at 0x10989e128>,
 4: <phue.Sensor object "Hue temperature sensor 1" at 0x10989e160>,
 5: <phue.Sensor object "Hall Sensor" at 0x10989e198>,
 6: <phue.Sensor object "Hue ambient light sensor 1" at 0x10989e0f0>,
 7: <phue.Sensor object "MotionSensor 5.Companion" at 0x10989e208>,
 8: <phue.Sensor object "Hue temperature sensor 2" at 0x109884b00>,
 9: <phue.Sensor object "Bedroom sensor" at 0x109884e80>,
 10: <phue.Sensor object "Hue ambient light sensor 2" at 0x1098a6240>,
 11: <phue.Sensor object "MotionSensor 9.Companion" at 0x1098a6278>,
 12: <phue.Sensor object "Robins iPhone" at 0x1098a62b0>,
 13: <phue.Sensor object "HomeAway" at 0x1098a62e8>,
 14: <phue.Sensor object "Hue temperature sensor 3" at 0x1098a6320>,
 15: <phue.Sensor object "Living room sensor" at 0x1098a6358>,
 16: <phue.Sensor object "Hue ambient light sensor

Probably accessing sensors via id is more useful than name. I need an init method that groups id by devices

# Lights

The HA component [adds the wrapper class](https://github.com/home-assistant/home-assistant/blob/aad14b8b87e23f61300f31b96b8a92769527516c/homeassistant/components/light/hue.py#L210) HueLight(Light).

Lights can be in groups and groups have info. I don't have any groups so access light objects directly

In [20]:
b.lights

[<phue.Light object "Hallway" at 0x1098cf6a0>,
 <phue.Light object "Bedroom light" at 0x1098cf390>,
 <phue.Light object "Lamp" at 0x1098cfba8>,
 <phue.Light object "Kitchen light" at 0x1098cfa20>]

In [21]:
def print_light_attrs(light_index):
    """Helper to print light attributes. Pass the index of light"""
    assert type(light_index) == int
    print("name : {}".format(b.lights[light_index].name))
    print("brightness : {}".format(b.lights[light_index].brightness))
    print("on : {}".format(b.lights[light_index].on))
    
light_index = 0
print_light_attrs(light_index)

name : Hallway
brightness : 1
on : False
